In [24]:
# Endpoints Ref: https://platform.worldcat.org/api-explorer/apis/wcapi

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import json

column_names = ['streaming','clio','wc_holdings','discard','title','bib_id','oclc_num',
                 'format','author','holdings_sup','bib_sup','tag_965','lp','historic_circs',
                     'comments','reserve_past_few','location']

# Passing in column names as a parameter in read_excel().
# Update inplace to avoid creating a new object (useful for larger dataframes).
# Fill in NaN values for new columns.

df = pd.read_excel("cat_data.xlsx", names = column_names)
df.dropna(subset = ['oclc_num'], inplace = True)
df.reset_index(drop = True, inplace = True)
df["clio_y_n"] = ""
df["num_libs"] = ""
df = df.astype({'oclc_num': 'int32'})

In [25]:
df.head()

,streaming,clio,wc_holdings,discard,title,bib_id,oclc_num,format,author,holdings_sup,bib_sup,tag_965,lp,historic_circs,comments,reserve_past_few,location,clio_y_n,num_libs
0,n,n,132.0,n,Bright Red [sound recording] / Laurie Anderson.,9062442,227189789,MUSICAL SOUND RECORDING,"Anderson, Laurie.",Y,Y,NaN,NaN,NaN,NaN,NaN,bar,,
1,NaN,NaN,NaN,NaN,"Chain : farm, factory, and the fate of our foo...",11043454,894893030,NONMUSICAL SOUND RECORDING,"Genoways, Ted,",Y,Y,NaN,NaN,NaN,NaN,NaN,bar,,
2,n,n,39.0,m,Egmont [sound recording] / Beethoven.,398584,80373612,MUSICAL SOUND RECORDING,"Beethoven, Ludwig van, 1770-1827.",Y,Y,NaN,y,NaN,Is this an LP??? We don't even have the capaci...,NaN,bar,,
3,y,y,733.0,y,Evolution [sound recording] / the United State...,4108195,44124520,MUSICAL SOUND RECORDING,United States Air Force Band.,Y,N,NaN,NaN,NaN,NaN,NaN,bar,,
4,n,n,1.0,n,Guido Cantelli [sound recording].,9162461,427008859,MUSICAL SOUND RECORDING,NaN,Y,Y,NaN,NaN,NaN,NaN,NaN,bar,,


In [26]:
#### Creating URLs####

API_Key = ""
OCLC_Sym = "ZCU"
base_url = "http://www.worldcat.org/webservices/catalog"

#urls for cul system
def create_clio_urls(oclc):
    search_endpoint = f"/content/libraries/{oclc}&oclcsymbol={OCLC_Sym}?wskey={API_Key}"
    url = base_url + search_endpoint
    return url

#urls for number of libraries
def create_lib_urls(oclc):
    search_endpoint = f"/content/libraries/{oclc}&frbrGrouping=off&maximumLibraries=100&format=json?wskey={API_Key}"
    url = base_url + search_endpoint
    return url

In [32]:
#### Actual API call, for now on a truncated version of the DF ####

#test df for api
#df_trun = df.head()

#api call to determine if oclc exists in the clio system and how many libraries have it

# We say that the holding exists in CLIO's catalogue if there is no message, because the message will say
# "Holding Not Found" if the holding doesn't exist in CLIO. Therefore, no message means it does exist.
def parse_response_clio(clio_url):
    clio_response = requests.get(clio_url)
    clio_soup = BeautifulSoup(clio_response.content,"lxml")
    if not clio_soup.message:
        df.at[index, 'clio_y_n'] = "y"
    else:
        df.at[index, 'clio_y_n'] = 'n'

def parse_response_lib(lib_url):
    lib_response  = requests.get(lib_url)
    lib_soup  = BeautifulSoup(lib_response.content,"html.parser")
    lib_soup_str = str(lib_soup)
    lib_soup_raw = lib_soup_str.replace("\\",r"\\")
    lib_soup_json = json.loads(lib_soup_raw)
    if ('totalLibCount' in lib_soup_json):
        df.at[index, 'num_libs'] = lib_soup_json['totalLibCount']
    else:
        df.at[index, 'num_libs'] = ''
    
    
for index, row in df.iterrows():
    clio_url = create_clio_urls(row['oclc_num'])
    lib_url = create_lib_urls(row['oclc_num'])
    parse_response_clio(clio_url)
    parse_response_lib(lib_url)


JSONDecodeError: Expecting ',' delimiter: line 1 column 12 (char 11)

In [11]:
df.head()

,streaming,clio,wc_holdings,discard,title,bib_id,oclc_num,format,author,holdings_sup,bib_sup,tag_965,lp,historic_circs,comments,reserve_past_few,location,clio_y_n,num_libs
0,n,n,132.0,n,Bright Red [sound recording] / Laurie Anderson.,9062442,227189789,MUSICAL SOUND RECORDING,"Anderson, Laurie.",Y,Y,NaN,NaN,NaN,NaN,NaN,bar,n,
1,NaN,NaN,NaN,NaN,"Chain : farm, factory, and the fate of our foo...",11043454,894893030,NONMUSICAL SOUND RECORDING,"Genoways, Ted,",Y,Y,NaN,NaN,NaN,NaN,NaN,bar,n,32
2,n,n,39.0,m,Egmont [sound recording] / Beethoven.,398584,80373612,MUSICAL SOUND RECORDING,"Beethoven, Ludwig van, 1770-1827.",Y,Y,NaN,y,NaN,Is this an LP??? We don't even have the capaci...,NaN,bar,n,8
3,y,y,733.0,y,Evolution [sound recording] / the United State...,4108195,44124520,MUSICAL SOUND RECORDING,United States Air Force Band.,Y,N,NaN,NaN,NaN,NaN,NaN,bar,y,708
4,n,n,1.0,n,Guido Cantelli [sound recording].,9162461,427008859,MUSICAL SOUND RECORDING,NaN,Y,Y,NaN,NaN,NaN,NaN,NaN,bar,y,1
